In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime as dt, timedelta
from dateutil.relativedelta import *
import re
from matplotlib import pyplot as plt
import seaborn as sns
import sqlite3

<h3>SQLite Handling functions</h3>
<p>Use Pandas to import dataframe into SQLlite</p>

In [2]:
DB = "..\\Data\\hockey_data_goalies.db"

#SQL Handle functions
def run_query(q):
    with sqlite3.connect(DB) as conn:
        x = pd.read_sql(q,conn)
    return x
   
#send command
def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.isolation_level = None
        return conn.execute(c)
    
#show tables
def show_tables():
    r = """
    Select name,type
    FROM sqlite_master
    WHERE type in ("table","view");
        """
    return run_query(r)

In [3]:
def sos_calc(year,game_no,nme,wind=[0]):
    """
        CALCULATES THE SOS FOR THE TEAM entered faced before a given date
    INPUT:
        year: the current season (YYYY-YYYY+1)
        game_no: the game_no for the current game previous games are [0-game_no-1]
        nme: the team abbr to calculate the SOS for
        RETURN:
        sos: the computed sos statistics sum(no_times_faced*(goalsfor-goalsagainst)/no_games)/no_games
    """
    
    sos = 0
    
    #get the details on the game
    q = ("""SELECT * 
            FROM team_log 
            WHERE team_id=\"{0}\"
            AND (CAST(SUBSTR(date_game,1,4) AS FLOAT)+CAST(SUBSTR(date_game,6,7) AS FLOAT)/12) > {1}
            AND (CAST(SUBSTR(date_game,1,4) AS FLOAT)+CAST(SUBSTR(date_game,6,7) AS FLOAT)/12) < {2}
            """.format(nme,int(year) + .66,int(year)+1.66))
    games = run_query(q)
    games['date_game'] = games['date_game'].astype('datetime64')
    
    gamedate = games.at[game_no,'date_game']

    if game_no>0:
        #list of teams faced to date (and number of times)
        if len(wind) ==1:
            teams_faced = games.loc[games['date_game']<gamedate,'opp_name'].value_counts()
        else:
            teams_faced = games.loc[wind,'opp_name'].value_counts()
       
        #get list of games played in the league that season up until the gameday
        q = ("""SELECT * 
                FROM team_log 
                WHERE (CAST(SUBSTR(date_game,1,4) AS FLOAT)+CAST(SUBSTR(date_game,6,7) AS FLOAT)/12) > {1}
                AND (CAST(SUBSTR(date_game,1,4) AS FLOAT)+CAST(SUBSTR(date_game,6,7) AS FLOAT)/12) < {2}
                """.format(nme,int(year) + .66,int(year)+1.66))
        temp_games = run_query(q)
        temp_games['date_game'] = temp_games['date_game'].astype('datetime64')

        #calculate statistics for each team
        goals = temp_games[temp_games['date_game']<gamedate].groupby('team_id').agg([sum,'count'])

        diff = (goals['goals']['sum']-goals['opp_goals']['sum'])/goals['goals']['count'] #calculates metric for all teams to date

        for eachteam in teams_faced.index:
            #get team_abbr

            q = '''SELECT team_abbr FROM team_list WHERE team_name=\"{0}\" AND CAST(SUBSTR(years_active,6,9) AS INT)>{1}'''.format(eachteam,int(year))
            nme = (run_query(q)['team_abbr'].values)[0]
            sos = sos + teams_faced[eachteam]*diff[nme]
        if len(wind) == 1:
            return sos/game_no
        else:
            return sos/wind.sum()
    else:
        return 0
    
def team_stat_gen(team_value,date):
    """
        Generates team based statistics for the season (and the last 3 weeks) upto the selected game
        INPUT:
            team_value: The team abbr
            year_value: The season in question where year is YYYY:YYYY+1
            game_value: The game_no for the current game previous games are [0-game_no-1] # must be more than 10
        OUTPUT:
            returns a data series of team stats
    """
    
    if date.month > 8:
        year_value = date.year
    else:
        year_value = date.year-1


    q = """SELECT game_number FROM team_log WHERE date_game=\"{0}\" AND team_id=\"{1}\"""".format(date.strftime("%Y-%m-%d"),team_value)
    game_value=run_query(q)['game_number'][0]
        
    
    #get the details on the game
    q = ("""SELECT * 
            FROM team_log 
            WHERE team_id=\"{0}\"
            AND (CAST(SUBSTR(date_game,1,4) AS FLOAT)+CAST(SUBSTR(date_game,6,7) AS FLOAT)/12) > {1}
            AND (CAST(SUBSTR(date_game,1,4) AS FLOAT)+CAST(SUBSTR(date_game,6,7) AS FLOAT)/12) < {2}
            """.format(team_value,int(year_value) + .66,int(year_value)+1.66))
    game_details = run_query(q)
    game_details['date_game'] = game_details['date_game'].astype('datetime64')
    try:
        game_date = game_details.at[game_value,'date_game']
    except:
        print(game_details,team_value,year_value,game_value)
        
    rest_days = game_date-game_details.at[game_value-1,'date_game']

    #team stats up to this point of the season (basic stats)
    cum_total = game_details.loc[game_details.index<game_value,['goals','opp_goals','shots','shots_against']].sum()
    games = (game_details.loc[game_details.index<game_value,'game_outcome']=='W').count() 
    wins = (game_details.loc[game_details.index<game_value,'game_outcome']=='W').sum()
    losses = (game_details.loc[game_details.index<game_value,'game_outcome']=='L').sum()
    ties = (game_details.loc[game_details.index<game_value,'game_outcome']=='T').sum()
    otl = ((game_details.loc[game_details.index<game_value,'game_outcome']=='L')&(game_details['overtimes'].notnull())).sum()
    points = wins*2+ties+otl
    points_pct = np.round(points/(2*games),3)
    sos = sos_calc(year_value,game_value,team_value) #strength of schedule higher is tougher
    srs = (cum_total['goals']-cum_total['opp_goals'])/games+sos #simple rating system
    return srs

<h2>Stat Functions</h2>
<p>Given the player statistics calculate:
    <ul>
        <li>Age</li>
        <li>Total minutes this season</li>
        <li>Total minutes last 3 weeks</li>
        <li>shots_against this season</li>
        <li>shots_against 3 weeks</li>
        <li>save_pct this season</li>
        <li>save_pct 3 weeks</li>
        <li>rest days</li>
       
</ul></p>

In [4]:
def season_finder(date):
    if date.month > 8:
        return date.year
    else:
        return date.year-1
def season_cumul(player):
    """ 
    Given the player, generate the career_log_data
    """
     #get the gamelogs for that player
    q = """SELECT * FROM player_log WHERE player_id=\"{0}\" """.format(player)
    player_logs = run_query(q)
    #unit conversions
    player_logs['date_game'] = player_logs['date_game'].astype('datetime64') #convert to datetime
    player_logs['time_on_ice'] = player_logs['time_on_ice'].str.extract(r'(\d*)\:\d*')[0].astype(int)+player_logs['time_on_ice'].str.extract(r'\d*\:(\d*)')[0].astype(int)/60
    player_logs['season'] = player_logs['date_game'].map(lambda x: season_finder(x))
    
    prodf = pd.DataFrame(columns=['player_id','team_id','opp_id','date_game','age','rest_days','min_season',
                                    'shots_against','save_pct','min3W','sa3W','svepct3W','future_save_pct','injured'])
    row = 0
    #for each season
    for season in player_logs['season'].unique():
        season_logs = player_logs[player_logs['season']==season].copy().reset_index(drop=True)
        for r,game in season_logs[10:].iterrows():
            prodf.loc[row,'player_id'] = season_logs.loc[r,'player_id']
            prodf.loc[row,'team_id'] = season_logs.loc[r,'team_id']
            prodf.loc[row,'team_srs'] = team_stat_gen(season_logs.at[r,'team_id'],season_logs.at[r,'date_game'])
            prodf.loc[row,'opp_id'] = season_logs.loc[r,'opp_id']
            prodf.loc[row,'opp_srs'] = team_stat_gen(season_logs.at[r,'opp_id'],season_logs.at[r,'date_game'])
            prodf.loc[row,'date_game'] = season_logs.loc[r,'date_game']
            prodf.loc[row,'age'] = season_logs.loc[r,'age']
            prodf.loc[row,'rest_days'] = (season_logs.loc[r,'date_game']-season_logs.loc[r-1,'date_game']).days
            prodf.loc[row,'min_season'] = season_logs.loc[:(r-1),'time_on_ice'].sum()
            prodf.loc[row,'shots_against'] = season_logs.loc[:(r-1),'shots_against'].sum()
            prodf.loc[row,'save_pct'] = np.round(season_logs.loc[:(r-1),'saves'].sum()/season_logs.loc[:(r-1),'shots_against'].sum(),3)
            
            window = (season_logs.loc[r,'date_game']>season_logs['date_game'])&(season_logs['date_game']>(season_logs.loc[r,'date_game']-timedelta(21)))
                        
            prodf.loc[row,'min3W'] = season_logs.loc[window,'time_on_ice'].sum()
            prodf.loc[row,'sa3W'] = season_logs.loc[window ,'shots_against'].sum()
            prodf.loc[row,'svepct3W'] = season_logs.loc[window,'saves'].sum()/season_logs.loc[window,'shots_against'].sum()
            prodf.loc[row,'future_save_pct'] = np.round(season_logs.loc[r,'saves'].sum()/season_logs.loc[r,'shots_against'].sum(),3)
            prodf.loc[row,'injured'] = season_logs.loc[r,'injured']
            row+=1
     
    return prodf

In [5]:
#for each player in the database
players = run_query('Select unique_id FROM player_list')
pdf = pd.DataFrame(columns=['player_id','team_id','opp_id','date_game','age','rest_days','min_season',
                                    'shots_against','save_pct','min3W','sa3W','svepct3W','future_save_pct','injured'])
for player in players['unique_id']:

    pdf = pdf.append(season_cumul(player))

pdf = pdf.fillna(pdf['save_pct'].mean())

C:\Users\jesse\anaconda3\envs\takeaseat\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\jesse\anaconda3\envs\takeaseat\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in longlong_scalars


IndexError: index out of bounds

In [ ]:
pdf.to_csv('advancedgoaliestats.csv')